# Shift the doctor

To be able to run notebook, generate an `doctor_input.csv` file in the same diretory with the format `name, assistant, part time, vacation, shift wishes, non-available times, last_shift` and added values for each doctor in a single line.

To generate the shift paln for a month, set Parameters `PLAN_YEAR` and `PLAN_MONTH` accordingly and run the notebook.

Output is `shift.xlsx` in same directory which provides a suggestion for the shift plan.

In [1]:
PLAN_YEAR = 2021
PLAN_MONTH = 10

In [2]:
import calendar
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_TIME, "de_DE")
import numpy as np
from pandas import ExcelWriter
import pandas as pd

In [16]:
doctors = pd.read_csv("doctor_input.csv", sep=";")

In [17]:
doctors

,name,assistant,part time,vacation,shift wishes,non-available times,last_shift
0,Schilling,1,100%,01.10.2021-10.10.2021,NaN,24.10.2021,25.09.2021
1,XY,2,80%,NaN,"12.10.2021,21.10.2021",NaN,27.09.2021
2,YZ,1,100%,NaN,NaN,13.10.2021,20.09.2021
3,AZ,2,100%,NaN,NaN,NaN,23.09.2021


In [14]:
def flatten(t):
    return [item for sublist in t for item in sublist]

doctors["part time"] = doctors["part time"].apply(lambda x: int(x.replace("%",""))/100)
doctors["last_shift"] = pd.to_datetime(doctors["last_shift"], format="%d.%m.%Y")
doctors["shift wishes"] = doctors["shift wishes"].apply(lambda x: [pd.to_datetime(datetime.strptime(d, "%d.%m.%Y")) for d in x.split(",")] if type(x) == str else x)
doctors["non-available times"] = doctors["non-available times"].apply(lambda x: [pd.to_datetime(datetime.strptime(d, "%d.%m.%Y")) for d in x.split(",")] if type(x) == str else x)
doctors["vacation"] = doctors["vacation"].apply(lambda x_list: flatten([pd.date_range(start=datetime.strptime(x.split("-")[0], "%d.%m.%Y"),end=datetime.strptime(x.split("-")[1], "%d.%m.%Y")).to_list() if type(x_list) != float else [x_list] for x in (x_list.split(",") if type(x_list) != float else [x_list])]))

In [6]:
month = calendar.month(PLAN_YEAR, PLAN_MONTH)
print(f"Generation month: {month}")
shift_plan = pd.DataFrame({"Date": pd.date_range(f"{PLAN_YEAR}-{PLAN_MONTH}-01", f"{PLAN_YEAR}-{PLAN_MONTH}-{calendar.monthrange(PLAN_YEAR, PLAN_MONTH)[1]}")})
shift_plan["Day"] = shift_plan.Date.dt.day_name()
shift_plan["1. Assistant"] = ""
shift_plan["2. Assistant"] = ""
shift_plan = shift_plan.set_index("Date")

Generation month:     Oktober 2021
Mo Di Mi Do Fr Sa So
             1  2  3
 4  5  6  7  8  9 10
11 12 13 14 15 16 17
18 19 20 21 22 23 24
25 26 27 28 29 30 31



In [7]:
for assistant in range(1,3):
    assistant_doctors = doctors.loc[doctors['assistant'] == assistant]
    for index, row in assistant_doctors.iterrows():
        if type(row["shift wishes"]) == list:
            for shift_wish in row["shift wishes"]:
                if shift_wish in shift_plan.index and not shift_plan.at[shift_wish, f"{assistant}. Assistant"]:
                    shift_plan.at[shift_wish, f"{assistant}. Assistant"] = row["name"]
    for index, row in shift_plan.iterrows():
        if row[f"{assistant}. Assistant"]:
            assistant_doctors.loc[assistant_doctors.name == row[f"{assistant}. Assistant"], "last_shift"] = index
            continue
        indices = []
        for index2, row2 in assistant_doctors.iterrows():
            available = True
            if type(row2["vacation"]) == list:
                if index in row2["vacation"]:
                    available = False
            elif type(row2["non-available times"]) == list:
                if index in row2["non-available times"]:
                    available = False
            elif (index - row2["last_shift"]).days == 1 or shift_plan.loc[index+timedelta(days=1)][f"{assistant}. Assistant"] == row2["name"]:
                available = False
            if available:
                indices.append(index2)
        available_doctors = assistant_doctors.loc[assistant_doctors.index.isin(indices)].copy()
        if not available_doctors.empty:
            available_doctors["dist_to_last"] = (index - available_doctors["last_shift"]) * available_doctors["part time"]
            name = available_doctors.iloc[available_doctors["dist_to_last"].argmax()]["name"]
            assistant_doctors.loc[assistant_doctors.name == name, "last_shift"] = index
            row[f"{assistant}. Assistant"] = name
            if row["Day"] == "Friday" and not index+timedelta(days=2) in assistant_doctors[assistant_doctors.name == name]["vacation"] and not index+timedelta(days=2) in assistant_doctors[assistant_doctors.name == name]["non-available times"]:
                shift_plan.at[index+timedelta(days=2), f"{assistant}. Assistant"] = name
        else:
            row[f"{assistant}. Assistant"] = "No one available"
        

C:\Users\chris\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\chris\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
shift_plan

,Day,1. Assistant,2. Assistant
Date,,,
2021-10-01,Friday,YZ,AZ
2021-10-02,Saturday,YZ,XY
2021-10-03,Sunday,YZ,AZ
2021-10-04,Monday,YZ,XY
2021-10-05,Tuesday,YZ,AZ
2021-10-06,Wednesday,YZ,XY
2021-10-07,Thursday,YZ,AZ
2021-10-08,Friday,YZ,XY
2021-10-09,Saturday,YZ,AZ


In [10]:
writer = ExcelWriter('shift.xlsx')
shift_plan.to_excel(writer)
writer.save()